In [1]:
from src.dataset import CERDataset
from src.mlp import MLPLucas
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import skill_metrics as sm

dataset = CERDataset(central_eolica='Aguia', window=3, horizons=12)

/home/gonca/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'src.right_preprocess'

In [ ]:
def plot_taylor(refs: dict, predictions_dict: dict):

    models = list(predictions_dict.keys())
    colors = ['c', 'm', 'y', 'k', 'r', 'b', 'g']
    colors = colors[:len(models)]
    models = {model: color for model, color in zip(models, colors)}
    for idx, (model, pred_dict) in enumerate(predictions_dict.items()):
        taylor_stats = []
        name = model[0]
        if model.endswith('ND'):
            name = name + 'ND'
        for horizon, pred in pred_dict.items():
            taylor_stats.append(sm.taylor_statistics(pred, refs[name][int(horizon)], 'data'))

        sdev = np.array([taylor_stats[0]['sdev'][0]]+[x['sdev'][1]
                                                    for x in taylor_stats])
        crmsd = np.array([taylor_stats[0]['crmsd'][0]]+[x['crmsd'][1]
                                                        for x in taylor_stats])
        ccoef = np.array([taylor_stats[0]['ccoef'][0]]+[x['ccoef'][1]
                                                        for x in taylor_stats])

        # To change other params in the plot, check SkillMetrics documentation in
        # https://github.com/PeterRochford/SkillMetrics/wiki/Target-Diagram-Options
        if len(list(predictions_dict.keys())) != 1:
            if idx != len(list(predictions_dict.keys()))-1 or len(list(predictions_dict.keys())) == 1:
                sm.taylor_diagram(sdev, crmsd, ccoef, styleOBS='-',
                                colOBS='g', markerobs='o',
                                titleOBS='Observation',
                                markercolor=models[model])
            else:
                sm.taylor_diagram(sdev, crmsd, ccoef, styleOBS='-',
                                titleOBS='Observation',
                                colOBS='g', markerobs='o', markercolor=models[model],
                                overlay = 'on', markerLabel=models)
        else:
            sm.taylor_diagram(sdev, crmsd, ccoef, styleOBS='-',
                      colOBS='g', markerobs='o',
                      titleOBS='Observation', markercolor='c',
                      markerLabel=['placeholder']+[
                          k+1 for k, v in pred_dict.items()])

In [ ]:
dataset.set_type('train')
dataset.set_horizon(0)
train_loader_lucas = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=8)
input_example_lucas = next(iter(train_loader_lucas))[0]
input_size_lucas = input_example_lucas.shape[1] * input_example_lucas.shape[2]

mlp = MLPLucas(
    window_size=input_example_lucas.shape[1],
    n_comps=input_example_lucas.shape[2],
    horizons=1,
)


In [ ]:
trainer = Trainer(gpus=1, max_epochs=10)   
trainer.fit(mlp, train_dataloaders=train_loader_lucas)
dataset.set_type('test')
mlp = mlp.cpu()
X_test_lucas = dataset.samples
y_mlp = mlp(X_test_lucas).detach()/dataset.test_scaler.scale_
y = dataset.labels/dataset.test_scaler.scale_


In [ ]:
y_mlp = y_mlp.numpy()
y = y.numpy()

plt.plot(y)
plt.plot(y_mlp)
plt.legend(['Original', 'Cabral'])
plt.show()

In [ ]:
import pandas as pd
a = {'ori': y, 'cabral': y_mlp}
a = pd.DataFrame(a)
a.corr()

In [ ]:
preds = {}
preds['Cabral'] = {0: y_mlp}
refs = {key: {0: y} for key in ['C']}
plot_taylor(refs, preds)
plt.show()

In [ ]:
len(dataset)